# barrier_cert

Synthesize a probabilistic barrier certificate to get a lower bound on the probability that a target set is reached. 
## Mathematics

Consider the polynomial dynamics

$$
x(t+1) = f(x(t), t, w),  \quad  w \sim \mathcal N(0, \Sigma)
$$
    
and two sets

$$  
X = \{x : g(x) \geq 0 \} \\
X_0 = \{x : g_0(x) \geq 0\} \\
X_1 = \{x : g_1(x) \geq 0\}
$$

We want to find a lower bound on the property

$$
p_{01} = \mathbb{P} \left[ x(T) \in X_1 \; \mid \; x(0) \in X_0, x(t) \in X \right]
$$

Consider a certificate $B(x, t)$ satisfying

\begin{align}
    B(x, t) &\geq 0        \quad & x \in X  \\
    B(x, 0) &\leq \gamma   \quad & x \in X_0     \\
    B(x, T) &\geq 1        \quad & x \not \in X_1, x \in X\\
    \mathbb{E} \left[ B(x(t+1), t+1)) \; \mid \; x(t) \right]  &\leq B(x(t), t) + c  \quad & x \in X
\end{align}

It obviously follows that with $B(t) = B(x(t), t)$ the last constraint implies $\mathbb{E} \left[ B(T) \right] < B(0) + cT$. Since $B$ is positive we can use Markov's inequality

$$
\mathbb{P} \left[ B(T) \leq 1 \right] = 1 - \mathbb{P} \left[ B(T) \geq 1 \right] \geq 1 - \mathbb{E} [B(T)] \geq 1 - (B(0) + cT)
$$

If $A \impliedby B$ and $C \implies D$, then $\mathbb{P}(A \mid C) \geq \mathbb{P}(B \mid D)$. From the constraints we have that
$$
    x(T) \in X_1 \impliedby B(T) \leq 1 \\
    x(0) \in X_0 \implies B(0) \leq \gamma
$$
Therefore,
$$
p_{01} = \mathbb{P}\left[x(T) \in X_1 \; \mid \; x(0) \in X_0 \right] \geq \mathbb{P}\left[ B(T) \leq 1 \; \mid \; B(0) \leq \gamma \right] \geq 1 - (\gamma + cT)
$$

Thus we search for a barrier certificate that satisfies the conditions above while minimizing $\gamma + cT$.

In [1]:
from sympy.abc import x, y, w
import numpy as np
import scipy.sparse as sp
from posipoly.polynomial import Polynomial
from posipoly.polylintrans import PolyLinTrans

from posipoly.ppp import *

g  = Polynomial.from_sympy(1, [x,y])
g0 = Polynomial.from_sympy(1**2 - x**2 - y**2, [x,y])
g1 = Polynomial.from_sympy(1**2 - (x-1)**2 - y**2, [x,y])

# dynamics: 
#  x(t+1) = x(t) + 0.25 + w
#  y(t+1) = y(t)

ft_x = Polynomial.from_sympy(x + 1 + w, [x,y,w])
ft_y = Polynomial.from_sympy(y, [x,y,w])

T = 1
sigma = 4
tot_deg = 6

We synhesize a separate barrier for each discrete time step, i.e. $B(x,t) = B_t(x)$. Using the S procedure we can rewrite the problem above as

\begin{align}
    c + B_t(x) - \mathbb{E} [B_{t+1}(f(x, t, w)) ] - r_t(x) g(x)  & \geq 0, \qquad  t=0, ..., T-1 \\
    \gamma - B_0(x) - s_0(x) g_0(x)  & \geq 0, \\
    B_T(x) - 1 + s_1(x) g_1(x) - u(x)g(x)      & \geq 0, \\
    B_t(x) - v_t(x) g(x)             & \geq 0, \qquad  t=0, ..., T \\
    s_0(x), s_1(x), u(x) & \geq 0, \\
    r_t(x) & \geq 0,    \qquad  t=0, ..., T-1 \\
    v_t(x) & \geq 0,     \qquad  t=0, ..., T
\end{align}
Or by introducing additional variables $e_t. w_t$ and $q_0, q_1$
\begin{align}
    c + B_t(x) - \mathbb{E} [ B_{t+1}(f(x,t,w)) ] - r_t(x) g(x) - e_t(x) & = 0, \qquad     t=0, ..., T-1 \\
    \gamma - B_0(x) - s_0(x)g_0(x) - q_0(x) & = 0, \\
    B_T(x) + s_1(x)g_1(x) - u(x)g(x) - q_1(x) & = 1, \\
    B_t(x) - v_t(x) g(x) - w_t(x) & = 0,                       \qquad t=0, ..., T \\
    s_0(x), s_1(x), q_0(x), q_1(x), u(x) & \geq 0, \\
    r_t(x), e_t(x) & \geq 0,    \qquad  t=0, ..., T-1 \\
    v_t(x), w_t(x) & \geq 0,     \qquad  t=0, ..., T
\end{align}

Let the variable vector be
$$
\begin{bmatrix} 
   c & \gamma & B_0 & \cdots & B_T & r_0 & \cdots & r_{T-1} & e_0 & \cdots & e_{T-1} & v_0 & \cdots & v_{T} & w_0 & \cdots & w_{T} & q_0 & s_0 & q_1 & s_1 & u
\end{bmatrix}
$$
where the positive variables are in gram matrix format and the rest in coefficient format. For simplicity we set up data structures with info about coefficient length.

In [2]:
from posipoly.grlex import count_monomials_leq 
def numvar_gram(n, d):
    # number of variables to represent a gram polynomial of degree d in n vars
    num_mon = count_monomials_leq(n, int(ceil(float(d)/2)))
    return num_mon*(num_mon+1)//2
def numvar_coef(n, d):
    return count_monomials_leq(n, d)

deg_B = tot_deg   # need to subtract if deg f larger than 1
deg_s0 = tot_deg-g0.d
deg_s1 = tot_deg-g1.d
deg_u = tot_deg-g.d

coef_names = ['c', 'gamma'] \
              + ['B{}'.format(t) for t in range(T+1)] \
              + ['r{}'.format(t) for t in range(T)] \
              + ['e{}'.format(t) for t in range(T)] \
              + ['v{}'.format(t) for t in range(T+1)] \
              + ['w{}'.format(t) for t in range(T+1)] \
              + ['q0', 's0', 'q1', 's1', 'u']
coef_lengths = [1, 1] \
               + [numvar_coef(2, deg_B)] * (T+1) \
               + [numvar_gram(2, deg_u)] * (T) \
               + [numvar_gram(2, tot_deg)] * (T) \
               + [numvar_gram(2, deg_u)] * (T+1) \
               + [numvar_gram(2, tot_deg)] * (T+1) \
               + [numvar_gram(2, tot_deg), numvar_gram(2, deg_s0), 
                  numvar_gram(2, tot_deg), numvar_gram(2, deg_s1),
                  numvar_gram(2, deg_u)]
coef_start = np.cumsum([0,] + coef_lengths[:-1])

# dict with name: (start, length)
coef_info = dict(zip(coef_names, zip(coef_start, coef_lengths) ))
numvar = sum(coef_lengths)

Prepare the transformation matrices

In [3]:
T1 = PolyLinTrans.eye(n0=1, n1=2, d0=0, d1=tot_deg).as_Tcc()

# B(x,y) -> B(fx(x,y,w), fy(x,y,w))
L_Bf_B = PolyLinTrans.composition(n0=2, d0=deg_B, g_list=[ft_x, ft_y])
# B(x,y,w) -> Bp(x,y) = E_w[ B(x,y,w) ]
L_exp = PolyLinTrans.gaussian_expectation(n0=L_Bf_B.n1, d0=L_Bf_B.d1, xi=2, sigma=sigma)
# B(x,y) -> E_w[ B(fx(x,y,w), fy(x,y,w)) ]
TBp = (L_exp * L_Bf_B).as_Tcc()
TB  = PolyLinTrans.eye(n0=2, n1=2, d0=deg_B).as_Tcc()         

Tg  = PolyLinTrans.mul_pol(n=2, d=deg_u, poly=g).as_Tcg()
Tg0 = PolyLinTrans.mul_pol(n=2, d=deg_s0, poly=g0).as_Tcg()
Tg1 = PolyLinTrans.mul_pol(n=2, d=deg_s1, poly=g1).as_Tcg()

Tfr = PolyLinTrans.eye(n0=2, n1=2, d0=tot_deg).as_Tcg()

These are the constraints in algebra form

    T1 c + TB Bt - Tbp B(t+1) - Tg rt -  Tfr et = 0   t=0, ..., T-1    (1)
    T1 gamma - TB B0 - Tg0 s0 - Tfr q0 = 0                             (2)
    TB BT + Tg1 s1 - Tg u - Tfr q1 = 1                                 (3)
    TB Bt - Tg vt - Tfr wt = 0                        t=0, ..., T      (4)
    
    vt, wt                      pp t=0, ..., T
    s0, s1, q0, q1, u           pp
    rt, et                      pp t=0, ..., T-1


In [4]:
Aeq = sp.coo_matrix((0, numvar))
beq = np.zeros(0)

numcon = T1.shape[0] # all constraints are in same set of monomials 
zero_multipliers = {name: sp.coo_matrix( (numcon, coef_info[name][1]) ) for name in coef_names}

# add (1)
for t in range(T):
    active_multipliers = {'c': T1,
                   'B{}'.format(t): TB, 
                   'B{}'.format(t+1): -TBp,
                   'r{}'.format(t): -Tg,
                   'e{}'.format(t): -Tfr}
    multipliers = {**zero_multipliers, **active_multipliers}    
    assert(all(name in coef_names for name in multipliers.keys()))
    Aeq = sp.bmat([[Aeq], [sp.bmat([[multipliers[name] for name in coef_names]])]])
    beq = np.hstack([beq, np.zeros(numcon)])

# add (2)
active_multipliers = {'gamma': T1, 'B0': -TB, 's0': -Tg0, 'q0': -Tfr}
multipliers = {**zero_multipliers, **active_multipliers}
assert(all(name in coef_names for name in multipliers.keys()))
Aeq = sp.bmat([[Aeq], [sp.bmat([[multipliers[name] for name in coef_names]])]])
beq = np.hstack([beq, np.zeros(numcon)])

# add (3)
active_multipliers = {'B{}'.format(T): TB, 's1': Tg1, 'u': -Tg, 'q1': -Tfr}
assert(all(name in coef_names for name in active_multipliers.keys()))
multipliers = {**zero_multipliers, **active_multipliers}
brow = np.zeros(numcon)
brow[0] = 1
Aeq = sp.bmat([[Aeq], [sp.bmat([[multipliers[name] for name in coef_names]])]])
beq = np.hstack([beq, brow])

# add (4)
for t in range(T+1):
    active_multipliers = {'B{}'.format(t): TB, 
                          'v{}'.format(t): -Tg,
                          'w{}'.format(t): -Tfr}
    multipliers = {**zero_multipliers, **active_multipliers}
    assert(all(name in coef_names for name in multipliers.keys()))
    Aeq = sp.bmat([[Aeq], [sp.bmat([[multipliers[name] for name in coef_names]])]])
    beq = np.hstack([beq, np.zeros(numcon)])

# inequality: gamma, c >= 0
Aiq = np.zeros((2, numvar))
Aiq[0,0] = -1  # c
Aiq[1,1] = -1  # gamma
biq = np.array([0, 0])
    
# objective
cost = np.zeros(numvar)
cost[0] = T  # weight on c
cost[1] = 1  # weight on gamma

# positivity constraints is everything except c and gamma
pp_names = ['r{}'.format(t) for t in range(T)] \
           + ['e{}'.format(t) for t in range(T)] \
           + ['v{}'.format(t) for t in range(T+1)] \
           + ['w{}'.format(t) for t in range(T+1)] \
           + ['q0', 's0', 'q1', 's1', 'u']
pp_list = [ list(coef_info[name]) for name in pp_names ]

Time to solve it!

In [8]:
sol, status = solve_ppp(cost, Aeq, beq, Aiq, biq, pp_list, 'sdd')

print('status:', status)

c = sol[0]
gamma = sol[1]
print('got c={:.2f} and gamma={:.2f}'.format(c, gamma))
print('lower bound is {:.2f}'.format(1-(gamma+c*T)))

status: solsta.optimal
got c=-0.04 and gamma=0.26
lower bound is 0.78


In [9]:
# check constraint satisfaction
if min(biq - Aiq.dot(sol)) < -1e-6:
    print('warning, iq constraint violated by {:f}'.format(abs(min(biq - Aiq.dot(sol)))))
    
if max(np.abs(Aeq.dot(sol)-beq)) > 1e-6:
    print('warning, eq constraint violated by {:f}'.format(max(np.abs(Aeq.dot(sol)-beq))) )

for a,b in pp_list:
    mat = vec_to_mat(sol[a:a+b])
    v, _ = np.linalg.eig(mat)
    if min(v) < -1e-6:
        print('warning, pp constraint violated by {:f}'.format(abs(min(v))))

warning, iq constraint violated by 0.042666
warning, eq constraint violated by 1.192063
warning, pp constraint violated by 0.000005
warning, pp constraint violated by 0.000006
